In [25]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from itertools import cycle
import numpy as np
from sklearn.metrics import (
    roc_curve,
    precision_recall_curve,
    auc,
    average_precision_score,
    f1_score,
    matthews_corrcoef)

In [26]:
######## Modify here
exp_path = './experiments/'
exp_ids = [
    'exp_100k_final_bs16_seed11_rmpssm0',
    'exp_100k_final_bs16_seed22_rmpssm0',
    'exp_100k_final_bs16_seed33_rmpssm0',
    'exp_100k_final_bs16_seed44_rmpssm0',
    'exp_100k_final_bs16_seed55_rmpssm0',
    'exp_100k_final_nostd_seed11_rmpssm_0',
    'exp_100k_final_nostd_seed22_rmpssm_0',
    'exp_100k_final_nostd_seed33_rmpssm_0',
    'exp_100k_final_nostd_seed44_rmpssm_0',
    'exp_100k_final_nostd_seed55_rmpssm_0',
    ]
exp_show_name=[
               'With Standardization_1',
               'With Standardization_2',
               'With Standardization_3',
               'With Standardization_4',
               'With Standardization_5',
               'Without Standardization_1',
               'Without Standardization_2',
               'Without Standardization_3',
               'Without Standardization_4',
               'Without Standardization_5']
comparison_id = 'standardize_experiment'
exp_type = 'shuffle'
exp_log = pd.read_excel(exp_path + '_experiments_log.xlsx', index_col='exp_id')
exp_log.head()

,exp_fullname,exp_path,start_time,end_time,input_data_path,protein_class,target_data,resolution,task,node_features,...,training_accuracy,validation_accuracy,testing_accuracy,training_precision,validation_precision,testing_precision,training_recall,validation_recall,testing_recall,test_clusters
exp_id,,,,,,,,,,,,,,,,,,,,,
exp_100k_final_feat_trans_seed11_rmpssm_0,exp_100k_final_feat_trans_seed11_rmpssm_0_230530,./experiments/exp_100k_final_feat_trans_seed11...,30/May/2023_02:50:52,30/May/2023_05:29:15,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.792,0.773,0.777,0.734,0.711,0.720,0.829,0.815,0.806,NaN
exp_100k_final_nostd_seed44_rmpssm_0,exp_100k_final_nostd_seed44_rmpssm_0_230529,./experiments/exp_100k_final_nostd_seed44_rmps...,29/May/2023_22:49:24,30/May/2023_03:26:25,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.764,0.754,0.747,0.715,0.690,0.702,0.770,0.798,0.739,NaN
exp_100k_final_nostd_seed55_rmpssm_0,exp_100k_final_nostd_seed55_rmpssm_0_230529,./experiments/exp_100k_final_nostd_seed55_rmps...,29/May/2023_22:49:43,30/May/2023_03:26:09,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.758,0.758,0.753,0.726,0.727,0.702,0.724,0.720,0.764,NaN
exp_100k_final_nostd_seed33_rmpssm_0,exp_100k_final_nostd_seed33_rmpssm_0_230529,./experiments/exp_100k_final_nostd_seed33_rmps...,29/May/2023_22:49:09,30/May/2023_03:24:54,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.753,0.755,0.760,0.772,0.777,0.733,0.624,0.622,0.715,NaN
exp_100k_final_nostd_seed11_rmpssm_0,exp_100k_final_nostd_seed11_rmpssm_0_230529,./experiments/exp_100k_final_nostd_seed11_rmps...,29/May/2023_22:48:45,30/May/2023_02:38:32,['/projects/0/einf2380/data/pMHCI/features_out...,I,BA,residue,classif,all,...,0.766,0.761,0.752,0.746,0.744,0.706,0.710,0.697,0.749,NaN


In [27]:
######## Definitions used in the plotting
comparisons_path = os.path.join(exp_path, 'comparisons')
comparison_path = os.path.join(comparisons_path, comparison_id)

if not os.path.exists(comparisons_path):
    os.makedirs(comparisons_path)

if not os.path.exists(comparison_path):
    os.makedirs(comparison_path)
else:
    print(f'Folder comparisons/{comparison_id}/ already exists! \
          \nChange comparison_id if you want to save plots for a different comparison.')

def get_single_exp_df(exp_id, exp_log, exp_path):
    exp_fullname = exp_log.loc[exp_id].exp_fullname
    exp_path = os.path.join(exp_path, exp_fullname)
    output_path = os.path.join(exp_path, 'output')
    output_train = pd.read_hdf(os.path.join(output_path, 'output_exporter.hdf5'), key='training')
    output_test = pd.read_hdf(os.path.join(output_path, 'output_exporter.hdf5'), key='testing')
    df = pd.concat([output_train, output_test])
    df.sort_values(by=['epoch'], inplace = True)
    return df

Folder comparisons/standardize_experiment/ already exists!           
Change comparison_id if you want to save plots for a different comparison.


In [28]:
#AUC score bar comparison(testing set)

palette = cycle(px.colors.qualitative.Plotly)

auc_scores = []
auc_scores_nostd = [0.8254,0.8335,0.8344,0.8225,0.8268]
auc_scores_std = [0.8515,0.8495,0.8522,0.8577,0.8506]
random_split=['Random_split1','Random_split2','Random_split3','Random_split4','Random_split5']


avg_nostd=sum(auc_scores_nostd)/len(auc_scores_nostd)
gap_avg_nostd=max(auc_scores_nostd)-avg_nostd
name_nostd=f'Without Standardization: AUC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(auc_scores_std)/len(auc_scores_std)
gap_avg_std=max(auc_scores_std)-avg_std
name_std=f'With Standardization: AUC={avg_std:.4f}±{gap_avg_std:.4f}'


fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = auc_scores_nostd),
    go.Bar(name= name_std, x=random_split, y = auc_scores_std)
])

fig.update_yaxes(title_text="AUC Score(Testing Set)")
fig.update_layout(
    barmode='group',
    title=f'AUC Scores for Data Standardization Experiment(Testing Set)', #modify
    title_x=0.5,
    width=700, height=600,
    yaxis_range=[0.80,0.89])
fig.write_html(os.path.join(comparison_path, 'std_auc_bars_testingset.html'))    


In [29]:
#AUC score bar comparison(validation set)


palette = cycle(px.colors.qualitative.Plotly)
random_split=['Random_split1','Random_split2','Random_split3','Random_split4','Random_split5']
auc_scores = []
auc_scores_nostd = [0.8341,0.8340,0.8379,0.8381,0.8316]
auc_scores_std = [0.8582,0.8587,0.8580,0.8605,0.8570]
random_split=['Random_split1','Random_split2','Random_split3','Random_split4','Random_split5']


avg_nostd=sum(auc_scores_nostd)/len(auc_scores_nostd)
gap_avg_nostd=max(auc_scores_nostd)-avg_nostd
name_nostd=f'Without Standardization: AUC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(auc_scores_std)/len(auc_scores_std)
gap_avg_std=max(auc_scores_std)-avg_std
name_std=f'With Standardization: AUC={avg_std:.4f}±{gap_avg_std:.4f}'


fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = auc_scores_nostd),
    go.Bar(name= name_std, x=random_split, y = auc_scores_std)
])

fig.update_yaxes(title_text="AUC Score(Validation Set)")
fig.update_layout(
    barmode='group',
    title=f'AUC Scores for Data Standardization Experiment(Validation Set)', #modify
    title_x=0.5,
    width=700, height=600,
    yaxis_range=[0.80,0.89])
fig.write_html(os.path.join(comparison_path, 'std_auc_bars_validationset.html'))

In [30]:
#AUC score bar comparison(training set)


palette = cycle(px.colors.qualitative.Plotly)
random_split=['Random_split1','Random_split2','Random_split3','Random_split4','Random_split5']
auc_scores = []
auc_scores_nostd = [0.8446,0.8422,0.8386,0.8430,0.8355]
auc_scores_std = [0.8737,0.8818,0.8796,0.8813,0.8770]
random_split=['Random_split1','Random_split2','Random_split3','Random_split4','Random_split5']


avg_nostd=sum(auc_scores_nostd)/len(auc_scores_nostd)
gap_avg_nostd=max(auc_scores_nostd)-avg_nostd
name_nostd=f'Without Standardization: AUC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(auc_scores_std)/len(auc_scores_std)
gap_avg_std=max(auc_scores_std)-avg_std
name_std=f'With Standardization: AUC={avg_std:.4f}±{gap_avg_std:.4f}'


fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = auc_scores_nostd),
    go.Bar(name= name_std, x=random_split, y = auc_scores_std)
])

fig.update_yaxes(title_text="AUC Score(Training Set)")
fig.update_layout(
    barmode='group',
    title=f'AUC Scores for Data Standardization Experiment(Training Set)', #modify
    title_x=0.5,
    width=700, height=600,
    yaxis_range=[0.80,0.89])
fig.write_html(os.path.join(comparison_path, 'std_auc_bars_trainingset.html'))

In [31]:
######## MCC bar plots(training set)
palette = cycle(px.colors.qualitative.Plotly)

mcc_nostd=[0.5279,0.5245,0.5188,0.5263,0.5119]
mcc_std=[0.5873,0.6030,0.5992,0.6029,0.5934]


thr = [0.50,0.49,0.50,0.49,0.50,
       0.50,0.51,0.52,0.52,0.49,
       ] #modify, max mcc threshold

avg_nostd=sum(mcc_nostd)/len(mcc_nostd)
gap_avg_nostd=max(mcc_nostd)-avg_nostd
name_nostd=f'Without Standardization: MCC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(mcc_std)/len(mcc_std)
gap_avg_std=max(mcc_std)-avg_std
name_std=f'With Standardization: MCC={avg_std:.4f}±{gap_avg_std:.4f}'

fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = mcc_nostd),
    go.Bar(name= name_std, x=random_split, y = mcc_std)
])

fig.update_yaxes(title_text="MCC Score(Training Set)")
fig.update_layout(
    barmode='group',
    title='MCC Scores for Data Standardization Experiment(Training Set)', #Modify
    title_x=0.5,
    width=800, height=500,
    yaxis_range=[0.49,0.6])
fig.write_html(os.path.join(comparison_path, 'std_mcc_thr_trainingset.html'))

In [32]:
######## MCC bar plots(validation set)
palette = cycle(px.colors.qualitative.Plotly)

mcc_nostd=[0.5136,0.5097,0.5200,0.5161,0.5076]
mcc_std=[0.5611,0.5554,0.5580,0.5609,0.5500]


thr = [0.56,0.50,0.53,0.58,0.45,
       0.58,0.51,0.50,0.47,0.45
       ] #modify, max mcc threshold

avg_nostd=sum(mcc_nostd)/len(mcc_nostd)
gap_avg_nostd=max(mcc_nostd)-avg_nostd
name_nostd=f'Without Standardization: MCC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(mcc_std)/len(mcc_std)
gap_avg_std=max(mcc_std)-avg_std
name_std=f'With Standardization: MCC={avg_std:.4f}±{gap_avg_std:.4f}'

fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = mcc_nostd),
    go.Bar(name= name_std, x=random_split, y = mcc_std)
])

fig.update_yaxes(title_text="MCC Score(Validation Set)")
fig.update_layout(
    barmode='group',
    title='MCC Scores for Data Standardization Experiment(Validation Set)', #Modify
    title_x=0.5,
    width=800, height=500,
    yaxis_range=[0.49,0.6])
fig.write_html(os.path.join(comparison_path, 'std_mcc_thr_validationset.html'))

In [33]:
######## MCC bar plots(testing set)
palette = cycle(px.colors.qualitative.Plotly)

mcc_nostd=[0.5014,0.5070,0.5117,0.4907,0.5053]
mcc_std=[0.5465,0.5390,0.5457,0.5498,0.5400]


thr = [0.57,0.57,0.64,0.50,0.54,
       0.47,0.56,0.61,0.39,0.58
       ] #modify, max mcc threshold

avg_nostd=sum(mcc_nostd)/len(mcc_nostd)
gap_avg_nostd=max(mcc_nostd)-avg_nostd
name_nostd=f'Without Standardization: MCC={avg_nostd:.4f}±{gap_avg_nostd:.4f}'

avg_std=sum(mcc_std)/len(mcc_std)
gap_avg_std=max(mcc_std)-avg_std
name_std=f'With Standardization: MCC={avg_std:.4f}±{gap_avg_std:.4f}'

fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=random_split, y = mcc_nostd),
    go.Bar(name= name_std, x=random_split, y = mcc_std)
])

fig.update_yaxes(title_text="MCC Score(Testing Set)")
fig.update_layout(
    barmode='group',
    title='MCC Scores for Data Standardization Experiment(Testing Set)', #Modify
    title_x=0.5,
    width=800, height=500,
    yaxis_range=[0.49,0.6])
fig.write_html(os.path.join(comparison_path, 'std_mcc_thr_testingset.html'))

In [34]:
######## Timings
# times_bs16=[]
# times_bs64=[]
# times_bs128=[]
# times_bs256=[]
# times_bs512=[]
# random_split=['Random_split1']
# for exp_id in exp_ids:
#     start = exp_log.loc[exp_id].start_time
#     end = exp_log.loc[exp_id].end_time
#     start_dt = datetime.strptime(start, '%d/%b/%Y_%H:%M:%S')
#     end_dt = datetime.strptime(end, '%d/%b/%Y_%H:%M:%S')
#     time = end_dt - start_dt

    
#     if "16" in exp_id:
#         times_bs16.append(int(time.seconds/60))
#     elif "64" in exp_id:
#         times_bs64.append(int(time.seconds/60))
#     elif "128" in exp_id:
#         times_bs128.append(int(time.seconds/60))
#     elif "256" in exp_id:
#         times_bs256.append(int(time.seconds/60))
#     elif "512" in exp_id:
#         times_bs512.append(int(time.seconds/60))
    

# avg_bs16=sum(times_bs16)/len(times_bs16)
# gap_avg_bs16=max(times_bs16)-avg_bs16
# name_bs16=f'Batch Size 16: Min={times_bs16[0]:.0f}'

# avg_bs64=sum(times_bs64)/len(times_bs64)
# gap_avg_bs64=max(times_bs64)-avg_bs64
# name_bs64=f'Batch Size 64: Min={times_bs64[0]:.0f}'

# avg_bs128=sum(times_bs128)/len(times_bs128)
# gap_avg_bs128=max(times_bs128)-avg_bs128
# name_bs128=f'Batch Size 128: Min={times_bs128[0]:.0f}'

# avg_bs256=sum(times_bs256)/len(times_bs256)
# gap_avg_bs256=max(times_bs256)-avg_bs256
# name_bs256=f'Batch Size 256: Min={times_bs256[0]:.0f}'

# avg_bs512=sum(times_bs512)/len(times_bs512)
# gap_avg_bs512=max(times_bs512)-avg_bs512
# name_bs512=f'Batch Size 512: Min={times_bs512[0]:.0f}'

# fig = go.Figure(data=[
#     go.Bar(name= name_bs16, x=random_split, y = times_bs16),
#     go.Bar(name= name_bs64, x=random_split, y = times_bs64),
#     go.Bar(name= name_bs128, x=random_split, y = times_bs128),
#     go.Bar(name= name_bs256, x=random_split, y = times_bs256),
#     go.Bar(name= name_bs512, x=random_split, y = times_bs512)
# ])

# fig.update_yaxes(title_text="Minutes")
# fig.update_layout(
#     barmode='group',
#     title='Total timings for the experiments',
#     title_x=0.5,
#     width=600, height=500)
# fig.write_html(os.path.join(comparison_path, 'timings.html'))

In [37]:
### Final comparison with best batch size & original batch size (AUC) ###
x_group=['Training Set','Validation Set','Testing Set']
auc_nostd=[0.8408,0.8351,0.8285]
auc_std=[0.8787,0.8585,0.8523]
name_nostd="Without Standardization (Original)"
name_std="With Standardization (Adopted)"
fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=x_group, y = auc_nostd, text = auc_nostd),
    go.Bar(name= name_std, x=x_group, y = auc_std, text = auc_std)
])

fig.update_yaxes(title_text="AUC Score")
fig.update_layout(
    barmode='group',
    title='Improvement of AUC Score for Data Standardization Experiment',
    title_x=0.5,
    width=800, height=600,
    yaxis_range=[0.80,0.89])
fig.write_html(os.path.join(comparison_path, 'final_std_auc.html'))

In [38]:
### Final comparison with best batch size & original batch size (MCC) ###
x_group=['Training Set','Validation Set','Testing Set']
mcc_nostd=[0.5219,0.5134,0.5032]
mcc_std=[0.5972,0.5571,0.5442]
name_nostd="Without Standardization (Original)"
name_std="With Standardization (Adopted)"
fig = go.Figure(data=[
    go.Bar(name= name_nostd, x=x_group, y = mcc_nostd, text = mcc_nostd),
    go.Bar(name= name_std, x=x_group, y = mcc_std, text = mcc_std)
])

fig.update_yaxes(title_text="MCC Score")
fig.update_layout(
    barmode='group',
    title='Improvement of MCC Score for Data Standardization Experiment',
    title_x=0.5,
    width=800, height=500,
    yaxis_range=[0.49,0.60])
fig.write_html(os.path.join(comparison_path, 'final_std_mcc.html'))